In [11]:
# needed to create the data frame
import pandas as pd

# create data frame from csv file we hosted on our github
df = pd.read_csv('https://raw.githubusercontent.com/uhi22018990/DataAnalytics/main/collision_data_clean_dnn.csv', index_col=0)

In [12]:
# make sure we have our data by printing it out
print(df) #all

      num_collisions  dim_date_year  dim_date_month  dim_date_day_number  \
1           0.152974           2012               7                    7   
2           0.314585           2012               7                    1   
3           0.936168           2012               7                    2   
4          -0.505904           2012               7                    3   
6           0.774556           2012               7                    5   
...              ...            ...             ...                  ...   
3765       -1.419631           2022              10                    5   
3766       -1.463141           2022              10                    6   
3767       -1.935544           2022              10                    7   
3768       -1.848523           2022              10                    1   
3769       -1.985271           2022              10                    2   

      temperature  dew_point  sea_level_pressure  visibility  wind_speed  \
1          

In [13]:
# needed to help with speedy maths based calculations
import numpy as np

# iloc allows us to select by rows. Here, we are shuffling the data by rows determined at random.
shuffle = df.iloc[np.random.permutation(len(df))]

# we are selecting all rows of the columns outlined excluding number of collisions
predictors = shuffle.iloc[:,1:12]

# print out the first 6 rows of predictors.
print(predictors[:6])

      dim_date_year  dim_date_month  dim_date_day_number  temperature  \
3705           2022               8                    1         75.2   
2627           2019               9                    1         70.5   
557            2014               1                    3         12.9   
1781           2017               5                    2         66.2   
3551           2022               3                    1         51.3   
332            2013               5                    2         61.1   

      dew_point  sea_level_pressure  visibility  wind_speed  \
3705       69.3              1013.9         9.5         2.8   
2627       57.3              1022.6        10.0         4.4   
557        -5.1              1031.1        10.0         6.3   
1781       44.1              1014.5        10.0         4.5   
3551       25.4              1016.3         9.6         8.6   
332        47.1              1022.2         7.1         2.3   

      max_sustained_wind_speed  max_temperatur

In [14]:
# print out the shuffled data (first 5 rows)
shuffle[:6]

,num_collisions,dim_date_year,dim_date_month,dim_date_day_number,temperature,dew_point,sea_level_pressure,visibility,wind_speed,max_sustained_wind_speed,max_temperature,min_temperature,total_precipitation
3705,-1.655832,2022,8,1,75.2,69.3,1013.9,9.5,2.8,7.0,82.0,72.0,0.0
2627,0.557002,2019,9,1,70.5,57.3,1022.6,10.0,4.4,8.9,78.1,63.0,0.0
557,0.637808,2014,1,3,12.9,-5.1,1031.1,10.0,6.3,11.1,21.0,9.0,0.0
1781,1.122643,2017,5,2,66.2,44.1,1014.5,10.0,4.5,8.0,79.0,53.1,0.0
3551,-1.376120,2022,3,1,51.3,25.4,1016.3,9.6,8.6,13.0,62.1,44.1,0.0
332,1.072916,2013,5,2,61.1,47.1,1022.2,7.1,2.3,7.0,66.9,55.4,0.0


In [15]:
# Select all rows for the num_collisions column
targets = shuffle.iloc[:,0]

# print out the first 6 rows of the targets data.
print(targets[:6])

3705   -1.655832
2627    0.557002
557     0.637808
1781    1.122643
3551   -1.376120
332     1.072916
Name: num_collisions, dtype: float64


In [16]:
# Define the scale.
SCALE_NUM_COLLISIONS = 1.0

In [17]:
# Split our data into a training set i.e. 80% of the length of the shuffle array
trainsize = int(len(shuffle['num_collisions'])*0.8)
# The test set size is 100% - 80% = 20% of the length of the shuffle array.
testsize = len(shuffle['num_collisions']) - trainsize

# Define the number of input values (predictors)
nppredictors = 11
# Define the number of output values (targets)
noutputs = 1

In [18]:
# let's see if the Google offical way works
%tensorflow_version 1.x
import tensorflow as tf
print(tf.__version__)

ValueError: ignored

In [19]:

## Google blocked also the hack used by the Tutor 
!pip install tensorflow==1.15.2
import tensorflow as tf

# check the version
print(tf.__version__)

# needed for high-level file management
import shutil  

# logging for tensorflow
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.INFO)

# removes a saved model from the last training attempt.
shutil.rmtree('/tmp/DNN_house_regression_trained_model', ignore_errors=True)

estimator = tf.contrib.learn.SKCompat(
    tf.contrib.learn.DNNRegressor(
        model_dir='/tmp/DNN_house_regression_trained_model', 
        hidden_units=[20,18,14], 
        optimizer=tf.train.AdamOptimizer(learning_rate=0.01), 
        enable_centered_bias=False, 
        feature_columns=tf.contrib.learn.infer_real_valued_columns_from_input(predictors.values)))

# Prints a log to show model is starting to train
print("starting to train");

# Train the model. Pass in predictor values and target values.
estimator.fit(
    predictors[:trainsize].values, targets[:trainsize].values.reshape(trainsize, noutputs) / SCALE_NUM_COLLISIONS, 
    steps=10000)

# Next, we can check our predictions based on our predictors.
preds = estimator.predict(x=predictors[trainsize:].values)

# Apply the Scale value (not really needed here) to the outputs.
predslistscale = preds['scores'] * SCALE_NUM_COLLISIONS

# pred = format(str(predslistscale)) # useful for checking outputs and printing.

# Calculate RMSE i.e. how good the model works using the predictions and targets.
# i.e. take the difference between the actual and the forecast then square the difference, 
# find the average of all the squares and then find the square root. 
# The RMSE essentially punishes larger errors i.e. it puts a heavier weight on larger errors.
rmse = np.sqrt(np.mean((targets[trainsize:].values - predslistscale)**2))
print('DNNRegression has RMSE of {0}'.format(rmse));


# Calculate the mean of the Life Satisfaction Values.
avg = np.mean(shuffle['num_collisions'][:trainsize])

# Calculate the RMSE using Life Satisfaction Values and the mean of all target values.
# The fit of a proposed regression model should therefore be better than the fit of the mean model.
# In this case, it doesn't seem to be the case but it will vary on every run.
rmse = np.sqrt(np.mean((shuffle['num_collisions'][trainsize:] - avg)**2))
print('Just using average = {0} has RMSE of {1}'.format(avg, rmse));

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
ERROR: Could not find a version that satisfies the requirement tensorflow==1.15.2 (from versions: 2.2.0, 2.2.1, 2.2.2, 2.2.3, 2.3.0, 2.3.1, 2.3.2, 2.3.3, 2.3.4, 2.4.0, 2.4.1, 2.4.2, 2.4.3, 2.4.4, 2.5.0, 2.5.1, 2.5.2, 2.5.3, 2.6.0rc0, 2.6.0rc1, 2.6.0rc2, 2.6.0, 2.6.1, 2.6.2, 2.6.3, 2.6.4, 2.6.5, 2.7.0rc0, 2.7.0rc1, 2.7.0, 2.7.1, 2.7.2, 2.7.3, 2.7.4, 2.8.0rc0, 2.8.0rc1, 2.8.0, 2.8.1, 2.8.2, 2.8.3, 2.8.4, 2.9.0rc0, 2.9.0rc1, 2.9.0rc2, 2.9.0, 2.9.1, 2.9.2, 2.9.3, 2.10.0rc0, 2.10.0rc1, 2.10.0rc2, 2.10.0rc3, 2.10.0, 2.10.1, 2.11.0rc0, 2.11.0rc1, 2.11.0rc2, 2.11.0)
ERROR: No matching distribution found for tensorflow==1.15.2
2.9.2


AttributeError: ignored

In [ ]:
#input data for testing estimator
dim_date_year 	dim_date_month
input = pd.DataFrame.from_dict(data = {
    'dim_date_year' : [2016, 2016, 2016, 2016, 2016, 2016, 2016, 2016, 2016, 2016],
    'dim_date_month' : [6, 6, 6, 6, 6, 6, 6, 6, 6, 6],
    'dim_date_day_number' : [5, 5, 5, 5, 5, 5, 5, 5, 5, 5],
    'temperature' : [47.3, 55.4, 36.3, 79.2, 77.5, 74.0, 70.9, 76.5, 80.0, 70.4],
    'dew_point'	 : [42.9, 44.4, 20.3, 57.7, 58.3, 62.4, 62.9, 64.5, 69.8, 49.5],
    'sea_level_pressure' 	 : [996.6, 1014.6, 1019.6, 1009.5, 1012.9, 1007.9, 1001.1, 1015.8, 1006.1, 1019.6],
    'visibility' : [7.1, 9.8, 10.0, 9.4, 10, 7.4, 7.9, 10, 9.8, 9],
    'wind_speed' : [9.4, 7.0, 4.2, 4.3, 4.2, 1.9, 4.6, 2.3, 2.5, 2],
    'max_sustained_wind_speed' : [32.1, 11.1, 7.0, 8.9, 8.0, 6.0, 8.9, 5.1, 6.0, 0.1],
    'max_temperature' : [55.9, 63.0, 43.0, 87.1, 82.9, 82.9, 78.1, 88.0, 86.0, 80.1],
    'min_temperature' : [39.9, 41.0, 25.0, 64.0, 71.1, 64.9, 64.9, 70.0, 66.0, 57.0],
    'total_precipitation' : [0.56, 0.01, 0.0, 0.0, 0.03, 0.68, 0.63, 0.0, 0.0, 0.0],
       })
					
# 2012-12-21 	5   751 	47.3 	42.9 	996.6 	7.1 	9.4 	32.1 	55.9 	39.0 	0.56
# 2013-12-21 	6   594 	55.4 	44.4 	1014.6 	9.8 	7.0 	11.1 	63.0 	41.0 	0.01	
# 2021-12-21 	2   256 	36.3 	20.3 	1019.6 	10.0 	4.2 	7 	  43.0 	25.0 	0.0
# 2016-06-21 	2   681 	79.2 	57.7 	1009.5 	9.4 	4.3 	8.9   87.1 	64.0  0.0
# 2017-06-21 	3   744 	77.5 	58.3 	1012.9 	10.0 	4.2 	8.0   82.9 	71.1  0.03
# 2018-06-21 	4   815 	74.0 	62.4 	1007.9 	7.4 	1.9 	6.0   82.9 	64.9  0.68
# 2019-06-21 	5   771 	70.9 	62.9 	1001.1 	7.9 	4.6 	8.9   78.1 	64.9  0.63
# 2020-06-21 	7   245 	76.5 	64.5 	1015.8 	10.0 	2.3 	5.1   88.0 	70.0  0.0
# 2021-06-21 	1   361 	80.0 	69.8 	1006.1 	9.8 	2.5 	6     86.0 	66.0  0.0
# 2022-06-21 	2   317 	70.4 	49.5 	1019.6 	9.0 	2 	  .1    80.1 	57.0  0.0


estimator = tf.contrib.learn.SKCompat(tf.contrib.learn.DNNRegressor(model_dir='/tmp/DNN_house_regression_trained_model', hidden_units=[20,18,14], enable_centered_bias=False, feature_columns=tf.contrib.learn.infer_real_valued_columns_from_input(input.values)))

preds = estimator.predict(x=input.values)

predslistnorm = preds['scores']
prednorm = format(str(predslistnorm))
print(prednorm)
